In [1]:
import sys
print(sys.version)


3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [9]:
pio.renderers.default = 'browser'

In [19]:
df_ple7= pd.read_csv(r"../data/data-1767790355598.csv")
df_ple1 = pd.read_csv(r"../data/data-1767790491093.csv")
df_data_raw = pd.read_csv(r"../data/data-raw30122025.csv", encoding='latin1')

df_ple1['maquina'] = 'Plegadora 1'
df_ple7['maquina'] = 'Plegadora 7'

df_ple1 = df_ple1.sort_values('temporal_placa').reset_index(drop=True)
df_ple7 = df_ple7.sort_values('temporal_placa').reset_index(drop=True)


In [ ]:
# Manejo de data-raw
import pandas as pd

df_data_raw.columns = (
    df_data_raw.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)

# Filtrar registros con texto 'PLEGADORA'
df_data = df_data_raw[
    df_data_raw['descripcion']
        .str.upper()
        .str.contains('PLEGADORA', na=False)
].copy()

# Extraer número de plegadora
df_data['nro_plegadora'] = (
    df_data['descripcion']
    .str.extract(r'PLEGADORA\s+(\d+)', expand=False)
)

# Quedarse solo con plegadora 1 y 7
df_data = df_data[df_data['nro_plegadora'].isin(['1', '7'])].copy()

df_data['maquina'] = 'Plegadora ' + df_data['nro_plegadora']

df_data[['descripcion', 'maquina']].drop_duplicates()


['nro_empresa', 'nro_orden_trabajo', 'nro_agenda', 'it_proceso', 'fecha_inicio', 'cod_puesto_trabajo', 'fecha_fin', 'cant_prog', 'cant_buenas', 'cant_malas', 'usuario_alta', 'fecha_alta', 'nro_agenda.1', 'cod_art_padre', 'cod_art', 'orden', 'id_chapa', 'herramental', 'oc_terc', 'ubicacion_almacen', 'almacen', 'cod_art_cli', 'usuario_prod', 'cod_puesto_trabajo.1', 'descripcion', 'dif_min', 'espesor', 'nombre', 'nro_vta', 'it_vta', 'unico', 'cant_usuario', 'fecha_usuario', 'usuario_user', 'nom_nesting', 'largo', 'ancho', 'peso', 'gui_nro_personas', 'gui_nro_golpes', 'ple_nro_personas', 'ple_nro_golpes', 'ple_camb_herra', 'tiempo_proc', 'nro_plegadora', 'maquina']


In [30]:
# Calculo inicio estimado del ciclo

# Duración del ciclo en minutos
df_data['duracion_minutos'] = pd.to_numeric(
    df_data['tiempo_proc'],
    errors='coerce'
)

df_data['fecha_fin'] = pd.to_datetime(
    df_data['fecha_usuario'],
    format='%d/%m/%Y %H:%M:%S:%f',
    errors='coerce'
)

df_data['fecha_inicio'] = (
    df_data['fecha_fin']
    - pd.to_timedelta(df_data['duracion_minutos'], unit='m')
)

# Ordenar cronológicamente por máquina
df_data = df_data.sort_values(
    by=['maquina', 'fecha_inicio']
).reset_index(drop=True)

# Crear un ID de ciclo por máquina
df_data['ciclo_id'] = (
    df_data
    .groupby('maquina')
    .cumcount() + 1
)

# DataFrame final de ciclos
df_ciclos = df_data[[
    'maquina',
    'ciclo_id',
    'fecha_inicio',
    'fecha_fin',
    'duracion_minutos'
]].copy()

df_ciclos.head()


,maquina,ciclo_id,fecha_inicio,fecha_fin,duracion_minutos
0,Plegadora 1,1,2025-12-29 23:33:15.733,2025-12-30 00:43:53.773,70.634
1,Plegadora 1,2,2025-12-30 06:14:04.773,2025-12-30 06:32:57.153,18.873
2,Plegadora 1,3,2025-12-30 07:12:23.447,2025-12-30 07:31:15.827,18.873
3,Plegadora 1,4,2025-12-30 07:29:15.670,2025-12-30 07:48:08.050,18.873
4,Plegadora 1,5,2025-12-30 07:53:23.680,2025-12-30 08:12:16.060,18.873


In [32]:
df_ciclos = df_ciclos.sort_values(
    by=['maquina', 'fecha_inicio']
)

df_ciclos


,maquina,ciclo_id,fecha_inicio,fecha_fin,duracion_minutos
0,Plegadora 1,1,2025-12-29 23:33:15.733000000,2025-12-30 00:43:53.773,70.634
1,Plegadora 1,2,2025-12-30 06:14:04.773000000,2025-12-30 06:32:57.153,18.873
2,Plegadora 1,3,2025-12-30 07:12:23.447000000,2025-12-30 07:31:15.827,18.873
3,Plegadora 1,4,2025-12-30 07:29:15.670000000,2025-12-30 07:48:08.050,18.873
4,Plegadora 1,5,2025-12-30 07:53:23.680000000,2025-12-30 08:12:16.060,18.873
5,Plegadora 1,6,2025-12-30 08:14:16.323000000,2025-12-30 08:33:08.703,18.873
6,Plegadora 1,7,2025-12-30 08:40:38.990000000,2025-12-30 08:57:53.090,17.235
7,Plegadora 1,8,2025-12-30 08:41:43.537000000,2025-12-30 08:58:57.637,17.235
8,Plegadora 1,9,2025-12-30 09:42:01.710000000,2025-12-30 09:58:36.510,16.580
9,Plegadora 1,10,2025-12-30 10:48:29.127000000,2025-12-30 11:05:23.607,16.908


In [5]:
def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )

    return df


In [6]:
df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

df_all = df_all.sort_values(['maquina','temporal_placa']).reset_index(drop=True)

In [7]:
# Visualización corrientes 
import plotly.graph_objects as go
import pandas as pd

# -------------------------
# Horarios
# -------------------------
fecha = df_all['temporal_placa'].dt.normalize().min()

horarios = {
    'TURNO_MAÑANA': (fecha + pd.Timedelta(hours=5),
                     fecha + pd.Timedelta(hours=17)),
    'ALMUERZO':     (fecha + pd.Timedelta(hours=12),
                     fecha + pd.Timedelta(hours=12, minutes=30)),
    'TURNO_TARDE':  (fecha + pd.Timedelta(hours=17),
                     fecha + pd.Timedelta(hours=22)),
    'CENA':         (fecha + pd.Timedelta(hours=22),
                     fecha + pd.Timedelta(hours=22, minutes=30)),
    'FUERA_TURNO':  (fecha + pd.Timedelta(hours=1),
                     fecha + pd.Timedelta(hours=5))
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(52, 101, 164, 0.25)',
    'TURNO_TARDE':  'rgba(46, 139, 87, 0.25)',
    'ALMUERZO':     'rgba(218, 165, 32, 0.30)',
    'CENA':         'rgba(178, 34, 34, 0.30)',
    'FUERA_TURNO':  'rgba(105, 105, 105, 0.20)'
}

# -------------------------
# Fases / máquinas
# -------------------------
fases = {
    'Fase R': 'corriente_r',
    'Fase S': 'corriente_s',
    'Fase T': 'corriente_t'
}

maquinas = list(df_all['maquina'].unique())

# -------------------------
# Figura y trazas
# -------------------------
fig = go.Figure()

labels = []  # para el dropdown

for maq in maquinas:
    g = df_all[df_all['maquina'] == maq]
    for fase_label, fase_col in fases.items():
        labels.append(f'{maq} – {fase_label}')

        fig.add_trace(
            go.Scatter(
                x=g['temporal_placa'],
                y=g[fase_col],
                mode='lines',
                name=f'{maq} – {fase_label}',
                visible=False
            )
        )

# Estado inicial
fig.data[0].visible = True

# -------------------------
# Dropdown único
# -------------------------
buttons = []

for i, label in enumerate(labels):
    visible = [False] * len(fig.data)
    visible[i] = True

    buttons.append(
        dict(
            label=label,
            method='update',
            args=[
                {'visible': visible},
                {'title.text': f'Corriente vs tiempo – {label}'}
            ]
        )
    )

# -------------------------
# Layout
# -------------------------
fig.update_layout(
    title=dict(
        text=f'Corriente vs tiempo – {labels[0]}',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    yaxis_title='Corriente [A]',
    showlegend=False,
    margin=dict(r=260),
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.00,
            xanchor='left',
            yanchor='top'
        )
    ]
)

# -------------------------
# Bandas horarias
# -------------------------
for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos[nombre],
        opacity=1,
        layer='below',
        line_width=0
    )

fig.show()


In [31]:
# Obtencion de franjas de inicio-fin de ciclos de trabajo
import plotly.graph_objects as go
import pandas as pd

# =========================================================
# 1. Determinar fase más cargada por MEDIA (por máquina)
# =========================================================
medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', 'mean'),
        I_S_media=('corriente_s', 'mean'),
        I_T_media=('corriente_t', 'mean')
    )
)

medias_por_fase['fase_mas_cargada'] = medias_por_fase[
    ['I_R_media', 'I_S_media', 'I_T_media']
].idxmax(axis=1)

map_fase_col = {
    'I_R_media': 'corriente_r',
    'I_S_media': 'corriente_s',
    'I_T_media': 'corriente_t'
}

map_fase_label = {
    'I_R_media': 'Fase R',
    'I_S_media': 'Fase S',
    'I_T_media': 'Fase T'
}

# =========================================================
# 2. Preparar figuras (una por máquina)
# =========================================================
figs = {}
maquinas = medias_por_fase.index.tolist()

for maq in maquinas:

    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    fase_col = map_fase_col[fase_media]
    fase_label = map_fase_label[fase_media]

    g_elec = df_all[df_all['maquina'] == maq]
    g_ciclos = df_ciclos[df_ciclos['maquina'] == maq]

    fig = go.Figure()

    # -----------------------------------------------------
    # Señal eléctrica: fase más cargada
    # -----------------------------------------------------
    fig.add_trace(
        go.Scatter(
            x=g_elec['temporal_placa'],
            y=g_elec[fase_col],
            mode='lines',
            line=dict(color='#1f77b4', width=2),
            name=f'{maq} – {fase_label}',
            hovertemplate=(
                'Tiempo: %{x}<br>'
                'Corriente RMS: %{y:.1f} A<br>'
                f'{fase_label}<extra></extra>'
            )
        )
    )

    # -----------------------------------------------------
    # Bandas de ciclos + líneas inicio/fin
    # -----------------------------------------------------
    for _, row in g_ciclos.iterrows():

        # Banda del ciclo
        fig.add_vrect(
            x0=row['fecha_inicio'],
            x1=row['fecha_fin'],
            fillcolor='rgba(0,0,0,0.06)',
            opacity=1,
            layer='above',
            line_width=0
        )

        # Línea inicio
        fig.add_vline(
            x=row['fecha_inicio'],
            line_color='rgba(0,0,0,0.25)',
            line_dash='dot',
            line_width=1
        )

        # Línea fin
        fig.add_vline(
            x=row['fecha_fin'],
            line_color='rgba(0,0,0,0.25)',
            line_dash='dot',
            line_width=1
        )

    # -----------------------------------------------------
    # Layout
    # -----------------------------------------------------
    fig.update_layout(
        title=dict(
            text=f'{maq} – {fase_label} (fase más cargada por media)',
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        xaxis_title='Tiempo',
        yaxis_title='Corriente RMS [A]',
        showlegend=False
    )

    figs[maq] = fig

# =========================================================
# 3. Dropdown para seleccionar máquina
# =========================================================
fig_master = go.Figure()

for i, maq in enumerate(maquinas):
    for tr in figs[maq].data:
        tr.visible = (i == 0)
        fig_master.add_trace(tr)

# Copiamos shapes (bandas + líneas) por máquina
fig_master.layout.shapes = figs[maquinas[0]].layout.shapes

buttons = []

offset = 0
for i, maq in enumerate(maquinas):
    n_traces = len(figs[maq].data)

    visible = [False] * len(fig_master.data)
    for j in range(offset, offset + n_traces):
        visible[j] = True

    buttons.append(
        dict(
            label=maq,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title.text': figs[maq].layout.title.text,
                    'shapes': figs[maq].layout.shapes
                }
            ]
        )
    )

    offset += n_traces

fig_master.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.0,
            xanchor='left',
            yanchor='top'
        )
    ],
    margin=dict(r=260)
)

fig_master.show()


In [ ]:
# Fase más cargada (por media)
import pandas as pd
import plotly.graph_objects as go

# ---------------------------------------------------------
# 1. Calcular medias por fase y máquina
# ---------------------------------------------------------
medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', 'mean'),
        I_S_media=('corriente_s', 'mean'),
        I_T_media=('corriente_t', 'mean')
    )
)

# Fase más cargada por máquina (según la media)
medias_por_fase['fase_mas_cargada'] = medias_por_fase[
    ['I_R_media', 'I_S_media', 'I_T_media']
].idxmax(axis=1)

# Mapeos auxiliares
map_fase_col = {
    'I_R_media': 'corriente_r',
    'I_S_media': 'corriente_s',
    'I_T_media': 'corriente_t'
}

map_fase_label = {
    'I_R_media': 'Fase R',
    'I_S_media': 'Fase S',
    'I_T_media': 'Fase T'
}

colores_fase = {
    'corriente_r': '#1f77b4',
    'corriente_s': '#2ca02c',
    'corriente_t': '#d62728'
}

# ---------------------------------------------------------
# 2. Construcción de la figura
# ---------------------------------------------------------
fig = go.Figure()

maquinas = list(medias_por_fase.index)

# Creamos UNA traza por máquina (solo su fase más cargada)
for i, maq in enumerate(maquinas):
    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    fase_col = map_fase_col[fase_media]
    fase_label = map_fase_label[fase_media]

    g = df_all[df_all['maquina'] == maq]

    fig.add_trace(
        go.Scatter(
            x=g['temporal_placa'],
            y=g[fase_col],
            mode='lines',
            name=f'{maq}',
            line=dict(
                color=colores_fase[fase_col],
                width=2
            ),
            visible=(i == 0),  # primera máquina visible
            hovertemplate=(
                'Tiempo: %{x}<br>'
                'Corriente RMS: %{y:.1f} A<br>'
                f'{fase_label} (fase más cargada)<extra></extra>'
            )
        )
    )

# ---------------------------------------------------------
# 3. Dropdown de selección de máquina
# ---------------------------------------------------------
buttons_maquina = []

for i, maq in enumerate(maquinas):
    visible = [False] * len(maquinas)
    visible[i] = True

    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    fase_label = map_fase_label[fase_media]

    buttons_maquina.append(
        dict(
            label=maq,
            method='update',
            args=[
                {'visible': visible},
                {'title.text': f'{maq} – {fase_label} (fase más cargada por media)'}
            ]
        )
    )

# ---------------------------------------------------------
# 4. Layout final
# ---------------------------------------------------------
fig.update_layout(
    title=dict(
        text=f'{maquinas[0]} – {map_fase_label[medias_por_fase.loc[maquinas[0], "fase_mas_cargada"]]} '
             '(fase más cargada por media)',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    xaxis_title='Tiempo',
    yaxis_title='Corriente RMS [A]',
    showlegend=False,
    margin=dict(r=220),
    updatemenus=[
        dict(
            buttons=buttons_maquina,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.00,
            xanchor='left',
            yanchor='top'
        )
    ]
)

fig.show()


In [22]:
# Visualizacion Tensiones
import plotly.graph_objects as go
import pandas as pd
fecha = df_all['temporal_placa'].dt.date.min()

horarios = {
    'TURNO_MAÑANA': (f'{fecha} 05:00', f'{fecha} 17:00'),
    'ALMUERZO':     (f'{fecha} 12:00', f'{fecha} 12:30'),
    'TURNO_TARDE':  (f'{fecha} 17:00', f'{fecha} 22:00'),
    'CENA':         (f'{fecha} 22:00', f'{fecha} 22:30'),
    'FUERA_TURNO':  (f'{fecha} 01:00', f'{fecha} 05:00')
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(52, 101, 164, 0.25)',   # azul acero
    'TURNO_TARDE':  'rgba(46, 139, 87, 0.25)',    # verde bosque
    'ALMUERZO':     'rgba(218, 165, 32, 0.30)',   # dorado apagado
    'CENA':         'rgba(178, 34, 34, 0.30)',    # rojo ladrillo
    'FUERA_TURNO':  'rgba(105, 105, 105, 0.20)'   # gris antracita
}

horarios = {
    k: (pd.to_datetime(v[0]), pd.to_datetime(v[1]))
    for k, v in horarios.items()
}

# Definición de fases
fases_tension = {
    'Fase R': 'tension_r',
    'Fase S': 'tension_s',
    'Fase T': 'tension_t'
}


maquinas = df_all['maquina'].unique()

fig = go.Figure()

# Colores por máquina (los mismos que corrientes)
colores_maquina = {
    'Plegadora 1': '#1f77b4',  # azul
    'Plegadora 7': '#d62728',  # rojo
}

maquinas = df_all['maquina'].unique()

# -----------------------------------------
# Trazas: fase × máquina (TENSION)
# -----------------------------------------
for fase_label, fase_col in fases_tension.items():
    for maq in maquinas:
        g = df_all[df_all['maquina'] == maq]

        fig.add_trace(
            go.Scatter(
                x=g['temporal_placa'],
                y=g[fase_col],
                mode='lines',
                name=f'{maq}',
                line=dict(
                    color=colores_maquina.get(maq, '#7f7f7f'),
                    width=2
                ),
                visible=(fase_label == 'Fase R')  # arranca en Fase R
            )
        )

# -----------------------------------------
# Dropdown de fases
# -----------------------------------------
buttons = []
n_maquinas = len(maquinas)

for i, fase_label in enumerate(fases_tension.keys()):
    visible = [False] * len(fig.data)

    for j in range(n_maquinas):
        visible[i * n_maquinas + j] = True

    buttons.append(
        dict(
            label=fase_label,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title': {
                        'text': f'Tensión vs tiempo – {fase_label}',
                        'x': 0.5,
                        'xanchor': 'center',
                        'font': {'size': 18}
                    }
                }
            ]
        )
    )

# -----------------------------------------
# Layout
# -----------------------------------------
fig.update_layout(
    title=dict(
        text='Tensión vs tiempo – Fase R',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    yaxis_title='Tensión [V]',
    xaxis_title='Tiempo',
    legend_title='Máquina',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.01,
            y=1.15
        )
    ]
)

# -----------------------------------------
# Bandas de turnos y pausas (idéntico a corrientes)
# -----------------------------------------
for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos.get(nombre, 'rgba(150,150,150,0.05)'),
        opacity=1,
        layer='below',
        line_width=0,
        annotation_text=nombre.replace('_', ' '),
        annotation_position='bottom left'
    )

fig.show()
